In [ ]:
#| default_exp core

# MarkdownMerge

> API details

In [ ]:
#| export
import os,mimetypes,smtplib

from fastcore.utils import *
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.policy import EmailPolicy
from email import encoders

from contextlib import contextmanager
from markdown import markdown
from email.headerregistry import Address
from email.header import Header
from time import sleep

In [ ]:
from fastcore.test import test_eq

In [ ]:
#| hide
from nbdev.showdoc import *

## Utility functions

In [ ]:
#| export
def get_addr(email, name=None):
    "Convert `email` and optional `name` into an email `Address` object"
    return Address(email if name is None else name, addr_spec=email)

Specify `from_addr` and `to_addrs` as either a string, or an `Address` object (created with `get_addr`). Note the `to_addrs` is a list.

In [ ]:
from_addr = get_addr('from@example.com', 'Jeremy Howard')
to_addrs = [get_addr('to@example.com', 'Jeręmy Hòwärd')]
print(from_addr)
print(to_addrs[0])

Jeremy Howard <from@example.com>
Jeręmy Hòwärd <to@example.com>


In [ ]:
#| export
def attach_file(msg, f):
    "Attach file `f` to message `msg`"
    mtype,_ = mimetypes.guess_type(f)
    main,sub = (mtype or 'application/octet-stream').split('/', 1)
    part = MIMEBase(main, sub)
    with open(f, 'rb') as fp: part.set_payload(fp.read())
    encoders.encode_base64(part)
    part['Content-Disposition'] = f'attachment; filename={Path(f).name}'
    msg.attach(part)

In [ ]:
msg = MIMEMultipart(policy=EmailPolicy())
attach_file(msg, '../pyproject.toml')
part = msg.get_payload()[0]
test_eq(part.get_content_type(), 'application/octet-stream')
test_eq(part['Content-Disposition'], 'attachment; filename=pyproject.toml')
assert len(part.get_payload())>20

In [ ]:
#| export
def create_multipart_msg(subj, from_addr, to_addrs, md=None, html=None, attach=None, hdrs=None):
    "Create a multipart email with markdown text and HTML"
    msg = MIMEMultipart('alternative', policy=EmailPolicy())
    msg['Subject'],msg['From'] = subj,str(from_addr)
    for k,v in (hdrs or {}).items(): msg[k]=v
    msg['To'] = ', '.join([str(a) for a in listify(to_addrs)])
    if md: msg.attach(MIMEText(md, 'plain'))
    if html: msg.attach(MIMEText(html, 'html'))
    for f in listify(attach): attach_file(msg, f)
    return msg

In [ ]:
msg = create_multipart_msg('Test Subject', from_addr, to_addrs, hdrs={'atest':'foo'},
                           md='**Bold text**', html='<b>Bold text</b>')
test_eq(msg['Subject'], 'Test Subject')
test_eq(msg['From'], str(from_addr))
test_eq(msg['atest'], 'foo')
test_eq(len(msg.get_payload()), 2)
test_eq(msg.get_payload()[0].get_content_type(), 'text/plain')
test_eq(msg.get_payload()[1].get_content_type(), 'text/html')
assert '@' in msg['To']

In [ ]:
#| export
def md2email(subj, from_addr, to_addrs, md, attach=None, hdrs=None):
    "Create a multipart email from markdown"
    html = markdown(md)
    return create_multipart_msg(subj, from_addr, to_addrs, md=md, html=html, attach=attach, hdrs=hdrs)

In [ ]:
test_msg2 = md2email('Test md2email', 'support@answer.ai', 'j@answer.ai', '**Markdown** test with _attachment_', attach='../pyproject.toml')
test_eq(test_msg2['Subject'], 'Test md2email')
payload = test_msg2.get_payload()
test_eq(len(payload), 3)
test_eq(payload[2]['Content-Disposition'], 'attachment; filename=pyproject.toml')

The basic email body is the plain text message (note that the template variables in `{}` will be filled in by `MarkdownMerge`):

In [ ]:
print(payload[0].get_payload())

**Markdown** test with _attachment_


Most email software is set up to display the HTML version:

In [ ]:
from IPython.display import HTML

In [ ]:
html = payload[1].get_payload()
HTML(html)

In [ ]:
att = payload[2].get_payload()

In [ ]:
import base64

In [ ]:
decoded = base64.b64decode(payload[2].get_payload())
print(decoded.decode('utf-8')[:35])

[build-system]
requires = ["setupto


In [ ]:
#| export
def smtp_connection(host, port, user=None, password=None, use_ssl=True, use_tls=False):
    "Create and return an SMTP connection"
    conn = smtplib.SMTP_SSL(host, port) if use_ssl else smtplib.SMTP(host, port)
    if use_tls and not use_ssl: conn.starttls()
    if user and password: conn.login(user, password)
    return conn

In [ ]:
servernm='email-smtp.us-west-2.amazonaws.com'
username=os.getenv('SES_SMTP_USER')
password=os.getenv('SES_SMTP_PASS')

In [ ]:
smtp_cfg = dict(host=servernm, port=587, user=username, password=password, use_ssl=False, use_tls=True)
test_msg = create_multipart_msg('Test from stdlib', 'support@answer.ai', 'j@answer.ai', md='**Test message**', html='<b>Test message</b>')
# try:
#     conn = smtp_connection(**smtp_cfg)
#     conn.send_message(test_msg)
# finally: conn.quit()

## MarkdownMerge -

In [ ]:
#| export
class MarkdownMerge:
    "Send templated email merge messages formatted with Markdown"
    def __init__(self, addrs, from_addr, subj, msg, smtp_cfg=None, inserts=None, test=False, hdrs=None, env_from=None, attach=None):
        self.addrs,self.from_addr,self.subj,self.msg,self.i = addrs,from_addr,subj,msg,0
        self.inserts = [{}]*len(addrs) if inserts is None else inserts
        self.smtp_cfg,self.test,self.hdrs,self.env_from,self.attach = smtp_cfg,test,hdrs,env_from,attach

    def send_msgs(self, pause=0.2):
        "Send all unsent messages to `addrs` with `pause` secs between each send"
        conn = smtp_connection(**self.smtp_cfg)
        while self.i < len(self.addrs):
            addr,insert = self.addrs[self.i],self.inserts[self.i]
            msg = self.msg.format(**insert)
            eml = md2email(self.subj, self.from_addr, addr, md=msg, hdrs=self.hdrs, attach=self.attach)
            if self.test: print(f"To: {addr}\n{'-'*40}\n{msg}\n{'='*40}\n")
            else:
                conn.send_message(eml, from_addr=self.env_from)
                sleep(pause)
            self.i += 1
            if self.i%100==0:
                print(self.i)
                conn.quit()
                conn = smtp_connection(**self.smtp_cfg)
        conn.quit()

    def reset(self): self.i=0

Your message should be in [markdown](https://daringfireball.net/projects/markdown/syntax) format. It will be converted into a two part email, containing both a plain text and an HTML part, so recipients will see whatever format they're set as their preference for viewing mail. Anything in curly brackets `{}` will be replaced with the contents of the inserts dictionary for that address. If there are no bracketed variables to replace, then you don't need to pass any inserts.

In [ ]:
msg = "**Hello {name}!**\n\nYour special number is: *{num}*"

`inserts` is a list of dictionaries. For each dictionary, the keys should match the bracketed names in your email template, and the values will be filled in to those sections.

In [ ]:
inserts = [{'name': 'Jeremy', 'num': 42}, {'name': 'Rachel', 'num': 7}]

In [ ]:
mm = MarkdownMerge(['aaa@answer.ai', 'bbb@answer.ai'], 'from@answer.ai', 'Test merge',
                   msg, smtp_cfg=smtp_cfg, inserts=inserts, test=True)

In [ ]:
#| eval: false
mm.send_msgs()

To: aaa@answer.ai
----------------------------------------
**Hello Jeremy!**

Your special number is: *42*

To: bbb@answer.ai
----------------------------------------
**Hello Rachel!**

Your special number is: *7*



Use `pause` to avoid sending too many messages too quickly; many SMTP servers restrict sending speed to avoid abuse. If you get an error during sending (e.g. "too many messages"), then wait an hour or so, then continue sending, using a larger `pause` value.

**NB**: You can just call `send_msgs` again when resending, since the successfully sent message count is saved, and those messages are not re-sent (unless you call `reset`). This includes test sends, therefore you should run reset after a test send.

To reset the counter to `0`, call `reset`:

In [ ]:
mm.reset()

### Attachments

Pass `attach` to include file attachments with your email. It can be a single file path or a list of paths:

```python
mm = MarkdownMerge(addrs, from_addr, 'Subject', msg, smtp_cfg=smtp_cfg, attach='report.pdf')
```

For multiple attachments:

```python
mm = MarkdownMerge(addrs, from_addr, 'Subject', msg, smtp_cfg=smtp_cfg, attach=['report.pdf', 'data.csv'])
```

The MIME type is automatically detected from the file extension. If the type cannot be determined, it defaults to `application/octet-stream`.